In [16]:
!pip install nerda -q
!pip install seqeval -q
!pip install nerda -q

In [17]:
from NERDA.datasets import get_conll_data, download_conll_data 
from google.colab import files
import pandas as pd

import unicodedata

import numpy as np
import seqeval.metrics
import spacy
import torch
from tqdm import tqdm, trange
from transformers import LukeTokenizer, LukeForEntitySpanClassification

uploaded = files.upload()


download_conll_data()
training = get_conll_data('train')
validation = get_conll_data('valid')


Saving retrain_processed.txt to retrain_processed.txt
Saving processed_df.csv to processed_df.csv
Saving retrain_processed.csv to retrain_processed.csv
Reading https://data.deepai.org/conll2003.zip


In [18]:
# Download the testb set of the CoNLL-2003 dataset
!wget https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testb

--2021-10-17 13:38:51--  https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 748096 (731K) [text/plain]
Saving to: ‘eng.testb.1’

eng.testb.1         100%[===================>] 730.56K  --.-KB/s    in 0.05s   

2021-10-17 13:38:52 (15.0 MB/s) - ‘eng.testb.1’ saved [748096/748096]



In [19]:
tag_scheme = [
'B-PER',
'I-PER',
'B-ORG',
'I-ORG',
'B-LOC',
'I-LOC',
'B-MISC',
'I-MISC'
]

In [20]:
def generate_labels(input_text):
  input_text = str(input_text)
  if input_text.count(" ") > 0:
    if "went to the store" in input_text:
      if input_text.count(" ") > 4:
        return ["B-PER", "I-PER", "O", "O", "O", "O"]
      return ["B-PER", "O", "O", "O", "O"]
    return ["B-PER", "I-PER"]
  else: 
    return ["B-PER"]

In [21]:
transformer = 'studio-ousia/luke-large-finetuned-conll-2003'

# model = LukeForEntitySpanClassification.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")
# model.eval()
# model.to("cuda")

In [22]:
# hyperparameters for network
dropout = 0.1

# hyperparameters for training
# training_hyperparameters = {
# 'epochs' : 5,
# 'warmup_steps' : 2500,                                                   
# 'train_batch_size': 2048,                                         
# 'learning_rate': 1e-5
# }

training_hyperparameters = {
'epochs' : 2,
'warmup_steps' : 500,                                                   
'train_batch_size': 13,                                         
'learning_rate': 1e-5
}

In [23]:
def get_sentence_from_name(input_name):
  input_name = str(input_name)
  return input_name.split(" ")

In [24]:
retrain_subset = pd.read_csv("retrain_processed.csv", index_col=0)
retrain_subset["tags_list"] = retrain_subset["Name"].apply(lambda x: generate_labels(x))
retrain_subset["sentences"] = retrain_subset["Name"].apply(lambda x: get_sentence_from_name(x))
# retrain_subset = retrain_subset[:100000]  
retrain_dict = {"sentences": list(retrain_subset["sentences"]), "tags": list(retrain_subset["tags_list"])}

In [25]:
total_sentences = list(retrain_dict["sentences"]) + list(training["sentences"])
total_tags = list(retrain_dict["tags"]) + list(training["tags"])

total_retrain_dict = {"sentences": total_sentences, "tags": total_tags}

# LUKE Model Finetuning

In [26]:
from NERDA.models import NERDA
model = NERDA(
dataset_training = total_retrain_dict,
dataset_validation = validation,
tag_scheme = tag_scheme, 
tag_outside = 'O',
transformer = transformer,
dropout = dropout,
hyperparameters = training_hyperparameters
)

Device automatically set to: cuda


Downloading:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

Some weights of the model checkpoint at studio-ousia/luke-large-finetuned-conll-2003 were not used when initializing LukeModel: ['classifier.weight', 'classifier.bias', 'luke.embeddings.position_ids']
- This IS expected if you are initializing LukeModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.6M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

In [ ]:
model.train()


 Epoch 1 / 2


 96%|█████████▋| 2006/2084 [18:35<00:43,  1.80it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #26112 length 154 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 30%|██▉       | 122/407 [00:14<00:33,  8.47it/s]

# Custom LUKE Model Testing on CoNLL 2003 Test Set


In [ ]:
test = get_conll_data('test')
model.evaluate_performance(test)

# Custom LUKE Model Testing on Curated Name Dataset


In [ ]:
processed_test_df = pd.read_csv("processed_df.csv", index_col=0)
processed_test_df["tags_list"] = processed_test_df["Name"].apply(lambda x: generate_labels(x))
processed_test_df["sentences"] = processed_test_df["Name"].apply(lambda x: get_sentence_from_name(x))
processed_test_dict = {"sentences": list(processed_test_df["sentences"]), "tags": list(processed_test_df["tags_list"])}


In [ ]:
model.evaluate_performance(processed_test_dict)

# Custom LUKE Model Testing on CoNLL 2003 Test Set


In [ ]:
# Load the tokenizer
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")

def load_documents(dataset_file):
    documents = []
    words = []
    labels = []
    sentence_boundaries = []
    with open(dataset_file) as f:
        for line in f:
            line = line.rstrip()
            if line.startswith("-DOCSTART"):
                if words:
                    documents.append(dict(
                        words=words,
                        labels=labels,
                        sentence_boundaries=sentence_boundaries
                    ))
                    words = []
                    labels = []
                    sentence_boundaries = []
                continue

            if not line:
                if not sentence_boundaries or len(words) != sentence_boundaries[-1]:
                    sentence_boundaries.append(len(words))
            else:
                items = line.split(" ")
                words.append(items[0])
                labels.append(items[-1])

    if words:
        documents.append(dict(
            words=words,
            labels=labels,
            sentence_boundaries=sentence_boundaries
        ))
        
    return documents


def load_examples(documents):
    examples = []
    max_token_length = 510
    max_mention_length = 30

    for document in tqdm(documents):
        words = document["words"]
        subword_lengths = [len(tokenizer.tokenize(w)) for w in words]
        total_subword_length = sum(subword_lengths)
        sentence_boundaries = document["sentence_boundaries"]

        for i in range(len(sentence_boundaries) - 1):
            sentence_start, sentence_end = sentence_boundaries[i:i+2]
            if total_subword_length <= max_token_length:
                # if the total sequence length of the document is shorter than the
                # maximum token length, we simply use all words to build the sequence
                context_start = 0
                context_end = len(words)
            else:
                # if the total sequence length is longer than the maximum length, we add
                # the surrounding words of the target sentence　to the sequence until it
                # reaches the maximum length
                context_start = sentence_start
                context_end = sentence_end
                cur_length = sum(subword_lengths[context_start:context_end])
                while True:
                    if context_start > 0:
                        if cur_length + subword_lengths[context_start - 1] <= max_token_length:
                            cur_length += subword_lengths[context_start - 1]
                            context_start -= 1
                        else:
                            break
                    if context_end < len(words):
                        if cur_length + subword_lengths[context_end] <= max_token_length:
                            cur_length += subword_lengths[context_end]
                            context_end += 1
                        else:
                            break

            text = ""
            for word in words[context_start:sentence_start]:
                if word[0] == "'" or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                text += word
                text += " "

            sentence_words = words[sentence_start:sentence_end]
            sentence_subword_lengths = subword_lengths[sentence_start:sentence_end]

            word_start_char_positions = []
            word_end_char_positions = []
            for word in sentence_words:
                if word[0] == "'" or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                word_start_char_positions.append(len(text))
                text += word
                word_end_char_positions.append(len(text))
                text += " "

            for word in words[sentence_end:context_end]:
                if word[0] == "'" or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                text += word
                text += " "
            text = text.rstrip()

            entity_spans = []
            original_word_spans = []
            for word_start in range(len(sentence_words)):
                for word_end in range(word_start, len(sentence_words)):
                    if sum(sentence_subword_lengths[word_start:word_end]) <= max_mention_length:
                        entity_spans.append(
                            (word_start_char_positions[word_start], word_end_char_positions[word_end])
                        )
                        original_word_spans.append(
                            (word_start, word_end + 1)
                        )

            examples.append(dict(
                text=text,
                words=sentence_words,
                entity_spans=entity_spans,
                original_word_spans=original_word_spans,
            ))

    return examples


def is_punctuation(char):
    cp = ord(char)
    if (cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False

In [ ]:
test_documents = load_documents("eng.testb")
test_examples = load_examples(test_documents)

In [ ]:
batch_size = 2
all_logits = []

for batch_start_idx in trange(0, len(test_examples), batch_size):
    batch_examples = test_examples[batch_start_idx:batch_start_idx + batch_size]
    texts = [example["text"] for example in batch_examples]
    entity_spans = [example["entity_spans"] for example in batch_examples]

    inputs = tokenizer(texts, entity_spans=entity_spans, return_tensors="pt", padding=True)
    inputs = inputs.to("cuda")
    with torch.no_grad():
        outputs = model(**inputs)
    all_logits.extend(outputs.logits.tolist())

In [ ]:
final_labels = [label for document in test_documents for label in document["labels"]]

final_predictions = []

for example_index, example in enumerate(test_examples):
    logits = all_logits[example_index]
    max_logits = np.max(logits, axis=1)
    max_indices = np.argmax(logits, axis=1)
    original_spans = example["original_word_spans"]
    predictions = []
    for logit, index, span in zip(max_logits, max_indices, original_spans):
        if index != 0:  # the span is not NIL
            predictions.append((logit, span, model.config.id2label[index]))

    # construct an IOB2 label sequence
    predicted_sequence = ["O"] * len(example["words"])
    for _, span, label in sorted(predictions, key=lambda o: o[0], reverse=True):
        if all([o == "O" for o in predicted_sequence[span[0] : span[1]]]):
            predicted_sequence[span[0]] = "B-" + label
            if span[1] - span[0] > 1:
                predicted_sequence[span[0] + 1 : span[1]] = ["I-" + label] * (span[1] - span[0] - 1)

    final_predictions += predicted_sequence

In [ ]:
print(seqeval.metrics.classification_report([final_labels], [final_predictions], digits=4)) 

# Baseline LUKE Model Testing on Curated Name Dataset


# Baseline LUKE Model Testing on CoNLL 2003 Dataset
